<a href="https://colab.research.google.com/github/J4sonN/MLprojects/blob/main/Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Complex Machine Learning Datasets

---

- Author: Quan Nguyen

---


#Dataset 1  and Machine Learning Model overview

- Data source: [Metabolic Syndrome](https://data.world/informatics-edu/metabolic-syndrome-prediction)  
- The dataset for analysis came from the NHANES initiative where the following variables were combined from multiple tables with SQL: abnormal waist circumference, triglycerides above 150, HDL cholesterol below 50 in women or 40 in men, history of hypertension and mildly elevated fasting blood sugar (100-125). Numerous other variables were added, such as uric acid, race, income, etc. that might contribute to the model but we will not sure, until we test the model.
- The target of this dataset is to predict the diagnosis of metabolic syndrome.
- One row represents a patient's info 
- This is a classification problem

In [1]:
#pandas
import pandas as pd
#numpy
import numpy as np 
#matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns

from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [2]:
ms = pd.read_csv('/content/drive/MyDrive/Projects/Metabolic  Syndrome.csv')
ms.head()

,seqn,Age,Sex,Marital,Income,Race,WaistCirc,BMI,Albuminuria,UrAlbCr,UricAcid,BloodGlucose,HDL,Triglycerides,MetabolicSyndrome
0,62161,22,Male,Single,8200.0,White,81.0,23.3,0,3.88,4.9,92,41,84,No MetSyn
1,62164,44,Female,Married,4500.0,White,80.1,23.2,0,8.55,4.5,82,28,56,No MetSyn
2,62169,21,Male,Single,800.0,Asian,69.6,20.1,0,5.07,5.4,107,43,78,No MetSyn
3,62172,43,Female,Single,2000.0,Black,120.4,33.3,0,5.22,5.0,104,73,141,No MetSyn
4,62177,51,Male,Married,NaN,Asian,81.1,20.1,0,8.13,5.0,95,43,126,No MetSyn


In [ ]:
print(f"There is {ms.shape[0]} rows, and {ms.shape[1]} columns in the data")

There is 2401 rows, and 15 columns in the data


##Data Cleaning

###Delete Unneccesary Columns

In [9]:
ms['seqn'].nunique()

2401

this columns has all different values it might be the identifying number of a patient, so we will drop this column.

In [12]:
ms.drop(columns=['seqn'], inplace=True)

###Delete Duplicated Rows

In [13]:
print(f'The dataset has {ms.duplicated().sum()} duplicated values')

The dataset has 0 duplicated values


###Handling missing value

In [15]:
ms.isna().sum()

Age                    0
Sex                    0
Marital              208
Income               117
Race                   0
WaistCirc             85
BMI                   26
Albuminuria            0
UrAlbCr                0
UricAcid               0
BloodGlucose           0
HDL                    0
Triglycerides          0
MetabolicSyndrome      0
dtype: int64

In [17]:
print(f'the data set has {ms.isna().sum().sum()} missing values')

the data set has 436 missing values


***There is no missing values on our target***

###Produce univariate visuals for the target and all features

#Dataset 2 and Machine Learning Model Overview

- Data Source: [California House Price](https://www.kaggle.com/datasets/shibumohapatra/house-price)
- The US Census Bureau has published California Census Data which has 10 types of metrics such as the population, median income, median housing price, and so on for each block group in California. The dataset also serves as an input for project scoping and tries to specify the functional and nonfunctional r requirements for it
- The target is to predict median house value of blocks
- One row represents all data of a block of houses 
- This is a regression problem.


In [6]:
ch = pd.read_csv('/content/drive/MyDrive/Projects/california-housing.csv')
ch.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,median_house_value
0,-122.23,37.88,41,880,129.0,322,126,8.3252,NEAR BAY,452600
1,-122.22,37.86,21,7099,1106.0,2401,1138,8.3014,NEAR BAY,358500
2,-122.24,37.85,52,1467,190.0,496,177,7.2574,NEAR BAY,352100
3,-122.25,37.85,52,1274,235.0,558,219,5.6431,NEAR BAY,341300
4,-122.25,37.85,52,1627,280.0,565,259,3.8462,NEAR BAY,342200


In [7]:
print(f"There is {ch.shape[0]} rows, and {ch.shape[1]} columns in the data")

There is 20640 rows, and 10 columns in the data
